In [1]:

import pandas as pd
import re

### Data Cleaning

#### Vehicle registration


In [2]:
url_2023 = "https://afdc.energy.gov/vehicle-registration?year=2023"
tables = pd.read_html(url_2023)
df_2023 = tables[0] 

url_2022 = "https://afdc.energy.gov/vehicle-registration?year=2022"
tables_2022 = pd.read_html(url_2022)
df_2022 = tables_2022[0]

url_2021 = "https://afdc.energy.gov/vehicle-registration?year=2021"
tables_2021 = pd.read_html(url_2021)
df_2021 = tables_2021[0]

url_2020 = "https://afdc.energy.gov/vehicle-registration?year=2020"
tables_2020 = pd.read_html(url_2020)
df_2020 = tables_2020[0] 

In [3]:
registration_2023 = df_2023.iloc[:, :2]
registration_2022 = df_2022.iloc[:, :2]
registration_2021 = df_2021.iloc[:, :2]
registration_2020 = df_2020.iloc[:, :2]

#combine all years into a single dataframe
all_registrations = pd.concat([registration_2023, registration_2022, registration_2021, registration_2020], axis=0)
all_registrations.columns = ['State', 'Registrations']
all_registrations['Year'] = [2023]*len(registration_2023) + [2022]*len(registration_2022) + [2021]*len(registration_2021) + [2020]*len(registration_2020)   
all_registrations


,State,Registrations,Year
0,Alabama,13000,2023
1,Alaska,2700,2023
2,Arizona,89800,2023
3,Arkansas,7100,2023
4,California,1256600,2023
...,...,...,...
47,Washington,50500,2020
48,West Virginia,600,2020
49,Wisconsin,6300,2020
50,Wyoming,300,2020


#### Charging ports

In [6]:
# Read the CSV and keep only State and Ports columns (trim header/footers as needed)
ports_2020 = pd.read_csv('/Users/wad/Documents/dsc190/project_repo/Datasets/2020_ports.csv', header=None, dtype=str)
ports_2021 = pd.read_csv('/Users/wad/Documents/dsc190/project_repo/Datasets/2021_ports.csv', header=None, dtype=str)
ports_2022 = pd.read_csv('/Users/wad/Documents/dsc190/project_repo/Datasets/2022_ports.csv', header=None, dtype=str)
ports_2023 = pd.read_csv('/Users/wad/Documents/dsc190/project_repo/Datasets/2023_ports.csv', header=None, dtype=str)

# The useful data rows start after the first 3 rows in this file; slice accordingly (adjust if necessary)
ports_2020 = ports_2020.iloc[3:107, [0, 4]].rename(columns={0: 'State', 4: 'Ports'})
ports_2021 = ports_2021.iloc[3:107, [0, 4]].rename(columns={0: 'State', 4: 'Ports'})
ports_2022 = ports_2022.iloc[3:107, [0, 4]].rename(columns={0: 'State', 4: 'Ports'})
ports_2023 = ports_2023.iloc[3:107, [0, 4]].rename(columns={0: 'State', 4: 'Ports'})

# Drop rows where State is missing/blank
ports_2020['State'] = ports_2020['State'].astype(str).str.strip()
ports_2020 = ports_2020[ports_2020['State'].notna() & (ports_2020['State'] != '')]

ports_2021['State'] = ports_2021['State'].astype(str).str.strip()
ports_2021 = ports_2021[ports_2021['State'].notna() & (ports_2021['State'] != '')]

ports_2022['State'] = ports_2022['State'].astype(str).str.strip()
ports_2022 = ports_2022[ports_2022['State'].notna() & (ports_2022['State'] != '')]

ports_2023['State'] = ports_2023['State'].astype(str).str.strip()
ports_2023 = ports_2023[ports_2023['State'].notna() & (ports_2023['State'] != '')]

# Extract the second number from the Ports column

def extract_second_number(s):
    if pd.isna(s):
        return pd.NA
    s = str(s)
    # split on '|' or '/' or ','
    parts = [p.strip() for p in re.split(r"\||/", s) if p.strip()!='']
    if len(parts) >= 2:
        num = parts[1].replace(',','')
        try:
            return int(num)
        except Exception:
            return pd.NA
    return pd.NA

ports_2020['Charging_Outlets'] = ports_2020['Ports'].apply(extract_second_number)

# Now keep only State and Charging_Outlets and aggregate by state (sum in case of continuation rows)
ports_2020_simple = (
    ports_2020.dropna(subset=['State'])
    .groupby('State', as_index=False)['Charging_Outlets']
    .sum(min_count=1)
)

ports_2021['Charging_Outlets'] = ports_2021['Ports'].apply(extract_second_number)

# Now keep only State and Charging_Outlets and aggregate by state (sum in case of continuation rows)
ports_2021_simple = (
    ports_2021.dropna(subset=['State'])
    .groupby('State', as_index=False)['Charging_Outlets']
    .sum(min_count=1)
)

ports_2022['Charging_Outlets'] = ports_2022['Ports'].apply(extract_second_number)

# Now keep only State and Charging_Outlets and aggregate by state (sum in case of continuation rows)
ports_2022_simple = (
    ports_2022.dropna(subset=['State'])
    .groupby('State', as_index=False)['Charging_Outlets']
    .sum(min_count=1)
)

ports_2023['Charging_Outlets'] = ports_2023['Ports'].apply(extract_second_number)

# Now keep only State and Charging_Outlets and aggregate by state (sum in case of continuation rows)
ports_2023_simple = (
    ports_2023.dropna(subset=['State'])
    .groupby('State', as_index=False)['Charging_Outlets']
    .sum(min_count=1)
)

In [7]:
all_ports = pd.concat([ports_2020_simple.assign(Year=2020),
                    ports_2021_simple.assign(Year=2021),
                    ports_2022_simple.assign(Year=2022),
                    ports_2023_simple.assign(Year=2023)],
                    axis=0)
all_ports

,State,Charging_Outlets,Year
0,Alabama,598,2020
1,Alaska,68,2020
2,Arizona,1788,2020
3,Arkansas,350,2020
4,California,34924,2020
...,...,...,...
48,Washington,5817,2023
49,West Virginia,446,2023
50,Wisconsin,1398,2023
51,Wyoming,252,2023
